# Identifiability of Structural Model for Lung Cancer Growth in Absence of Treatment

The tumour growth inhibitory effects of Erlotinib and Gefitinib were modelled with a population PKPD model in [1]. A population PKPD model is a hierarchical model which consists of a structural model, a population model, and an error model. Each sub-model captures a different aspect of the tumour growth inhibition biology, and is parametrised by a set of parameters. 

In this notebook we explore the identifiability of the structural model reported in [1] for modelling the tumour growth of LXF A677 implants in mice in absence of treatment. In particular, we investigate the effect of log-transforming and non-dimensionalisation of the model parameters on the stability of the inference. 

## Structural growth model in absence of treatment

In a PKPD model the structural model is a mechanistic or empirical description of the drug's pharmacokinetics (PK) and pharmacodynamics (PD). In other words, the PK model describes how the drug is distributed in the body upon administration, and in particular models the amount of the compound at the site of action over time. The PD model captures the drug's effects on the progression of the disease. 'Progression of the disease' may be quantified by any disease-related observable, such as biomarkers or in this case the tumour volume. As a result, a PD model does not only need to capture the disease progression under influence of the compound, but also needs to be able to describe its progresion in the absence of any treatment.

In [1] a structural model is proposed for the tumour growth in absence of the compound, which assumes that the tumour grows exponentially for small tumour volumes, and linearly for large tumour volumes

\begin{equation*}
    \frac{\text{d}V^s_T}{\text{d}t} = \frac{2\lambda _0\lambda _1 V^s_T}{2\lambda _0 V^s_T + \lambda _1}.
\end{equation*}

Here, $V^s_T$ is the predicted tumour volume by the structural model, $\lambda _0$ is the exponential growth rate, and $\lambda _1$ is the linear growth rate. The tumour growth of an individual in absence of the drug is thus parameterised by three parameters

\begin{equation*}
    \psi = (V_0, \lambda _0, \lambda _1),
\end{equation*}

where $V_0$ is the initial tumour volume, $V^s_T(t=0, \psi) = V_0$.

## Identifiability

The identifiability of parametric models is a multifacetted problem and has been addressed many times in the literature, e.g. [2, 3, 4]. Generally, identifiability of a model refers to the ability of inferring its parameters from data. In particular, under the assumption that the model is able to generate the data with a given set of model parameters, identifiability addresses the question of whether those parameters can be recovered from the data. In the PKPD modelling context, the identifiability of the structural model is of particular importance, as model parameters are often biologically interpretable. This allows to characterise biochemical properties of the compound based on the inferred model parameters, and even to translate PKPD models from preclinical to clinical application. 

In this study, we are interested to learn the structural model parameters $\psi $ of the tumour growth in absence of the compound from a *in vivo* experiment reported in [1]. Here, patient-derived tumour explants LXF A677 (adenocarcinoma of the lung) were implanted in mice. The tumour growth was monitored over a period of 30 days, see the [Control Growth Data notebook](https://nbviewer.jupyter.org/github/DavAug/ErlotinibGefitinib/blob/master/notebooks/control_growth/data_preparation.ipynb) for details.

In [59]:
#
# Visualise control growth data.
#

import os

import pandas as pd
import plotly.colors
import plotly.graph_objects as go


# Import data
# Get path of current working directory
path = os.getcwd()

# Import LXF A677 control growth data
data = pd.read_csv(path + '/data/lxf_control_growth.csv')

# Get mouse ids
mouse_ids = data['#ID'].unique()

# Get number of mice
n_mice = len(mouse_ids)

# Define colorscheme
colors = plotly.colors.qualitative.Plotly[:n_mice]

# Create figure
fig = go.Figure()

# Scatter plot LXF A677 time-series data for each mouse
for index, id_m in enumerate(mouse_ids):
    # Create mask for mouse
    mask = data['#ID'] == id_m

    # Get time points for mouse
    times = data['TIME in day'][mask]

    # Get observed tumour volumes for mouse
    observed_volumes = data['TUMOUR VOLUME in cm^3'][mask]

    # Get mass time series
    masses = data['BODY WEIGHT in g'][mask]

    # Plot tumour volume over time
    fig.add_trace(go.Scatter(
        x=times,
        y=observed_volumes,
        name="ID: %d" % id_m,
        showlegend=True,
        hovertemplate=
            "<b>ID: %d</b><br>" % (id_m) +
            "Time: %{x:} day<br>" +
            "Tumour volume: %{y:.02f} cm^3<br>" +
            "Body weight: %{text}<br>" +
            "Cancer type: Lung cancer (LXF A677)<br>" +
            "<extra></extra>",
        text=['%.01f g' % mass for mass in masses],
        mode="markers",
        marker=dict(
            symbol='circle',
            color=colors[index],
            opacity=0.7,
            line=dict(color='black', width=1))
    ))

    # Plot mass over time
    fig.add_trace(go.Scatter(
        x=times,
        y=masses,
        name="ID: %d" % id_m,
        showlegend=True,
        visible=False,
        hovertemplate=
            "<b>ID: %d</b><br>" % (id_m) +
            "Time: %{x:} day<br>" +
            "Tumour volume: %{y:.02f} cm^3<br>" +
            "Body weight: %{text}<br>" +
            "Cancer type: Lung cancer (LXF A677)<br>" +
            "<extra></extra>",
        text=['%.01f g' % mass for mass in masses],
        mode="markers",
        marker=dict(
            symbol='circle',
            color=colors[index],
            opacity=0.7,
            line=dict(color='black', width=1))
    ))

# Set X, Y axis and figure size
fig.update_layout(
    autosize=True,
    xaxis_title=r'$\text{Time in day}$',
    yaxis_title=r'$\text{Tumour volume in cm}^3$',
    template="plotly_white")

# Add switch between linear and log y-scale
fig.update_layout(
    updatemenus=[
        dict(
            type = "buttons",
            direction = "left",
            buttons=list([
                dict(
                    args=[{"yaxis.type": "linear"}],
                    label="Linear y-scale",
                    method="relayout"
                ),
                dict(
                    args=[{"yaxis.type": "log"}],
                    label="Log y-scale",
                    method="relayout"
                )
            ]),
            pad={"r": 0, "t": -10},
            showactive=True,
            x=0.0,
            xanchor="left",
            y=1.15,
            yanchor="top"
        ),
        dict(
            type = "buttons",
            direction = "down",
            buttons=list([
                dict(
                    args=[
                        {"visible": [True, False] * n_mice,},
                        {"yaxis": {"title": r'$\text{Tumour volume in cm}^3$'}}],
                    label="Tumour volume",
                    method="update"
                ),
                dict(
                    args=[
                        {"visible": [False, True] * n_mice}, 
                        {"yaxis": {"title": r'$\text{Body weight in g}$'}}],
                    label="Body weight",
                    method="update"
                ),
            ]),
            pad={"r": 0, "t": -10},
            showactive=True,
            x=1.07,
            xanchor="left",
            y=1.1,
            yanchor="top"
        ),
    ]
)

# Position legend
fig.update_layout(legend=dict(
    yanchor="bottom",
    y=0.01,
    xanchor="left",
    x=1.05))

# Show figure
fig.show()

**Figure 1:** Untreated tumour growth of patient-derived tumour explants LXF A677 (adenocarcinoma of the lung) that were implanted in mice. The colouring of the data points indicates that the measurements belong to the same mouse. Mouse ID and further information can be explored by hovering over the data points. The evolution of the body weight can be explored using the buttons in the top right.

## Naïve optimisation of model parameters

Let us begin the exploration of the identifiability of the model by attempting to find an optimal set of model parameters $\psi $ for each mouse. Typically such a set of optimal parameters can be found by defining an objective function and an optimisation algorithm. The objective function $L(\psi | V^{\text{obs}}_{T})$ quantifies how close the model predictions $V^s_T(t_i, \psi)$ for a given set of model parameters $\psi $ are to the observed data $V^{\text{obs}}_{T, i}$. There are many choices for objective funtions. We somewhat arbitrarily chose the Squared Distance Error Measure

\begin{equation*}
    L(\psi | V^\text{obs}_{T}) = \sum ^{n}_{i=1}\left( V^\text{obs}_{T, i} - V^s_T(t_i, \psi)\right) ^2,
\end{equation*}

where $n$ is the number of measurements, $V^\text{obs}_{T, i}$ is the measured tumour volume at time $t_i$ and $V^s_T(t_i, \psi)$ is the model prediction at time $t_i$ for model parameters $\psi $. This objective function is greater or equal to zero for all $\psi $, and only vanishes for a model parameter set $\hat \psi $ for which the model identically reproduces the observations. We would like to find this parameter set.

For real measurements, the measurement process alone will make it improbable that the model will be able to reproduce the observations exactly, as measurement noise introduces deviations from the true underlying process. As a result, we are forced to weaken our condition for an optimal parameter set and look for the set of parameters $\hat \psi $ that globally minimise the objective function $L$. Note that this step introduces uncertainty into the modelling process, not only because we can no longer decide whether a given set of parameters is optimal by just looking at its objective function score, but also because it's much harder to decide whether the deviations from the observations are due to noise or due to the wrong modelling choice.

Reserving the problem of model selection for later, there are many algorithms that are designed to find the global minimum of an objective function. One of those algorithms is the Covariance Matrix Adaptation Evolution Strategy (CMA-ES) optimiser [5]. We will choose to minimise the objective $L$ with this algorithm, but other choices are equally valid. From a naïve perspective, we can choose any initial starting point $\psi _0 $ for the optimisation algorithm, as the true global minimum is independent of the method we use to find it. We therefore choose to start the optimisation somewhat arbitrarily at $\psi _0 = (1, 1, 1)$.

In [5]:
#
# Create structural model.
#

import pkpd.model as m


# Create model
model = m.create_tumour_growth_model()

# Show model
print(model.code())

[[model]]
# Initial values
central.volume_t = 0

[central]
lambda_0 = 0
    in [1/day]
lambda_1 = 1
    in [cm^3/day]
time = 0 bind time
    in [day]
dot(volume_t) = 2 * (lambda_0 * (lambda_1 * volume_t)) / (2 * (lambda_0 * volume_t) + lambda_1)
    in [cm^3]




In [50]:
#
# Define pints model wrapper such that myokit model can be used for inference.
#

import myokit
import pints

from pkpd import model as model


# Wrap myokit model, so it can be used with pints
class PintsModel(pints.ForwardModel):
    def __init__(self):
        # Create myokit model
        model = m.create_tumour_growth_model()

        # Create simulator
        self.sim = myokit.Simulation(model)

    def n_parameters(self):
        """
        Number of parameters to fit. Here initial V^s_T, lambda_0, lambda_1
        """
        return 3

    def n_outputs(self):
        return 1

    def simulate(self, parameters, times):
        # Reset simulation
        self.sim.reset()

        # Sort input parameters
        initial_volume, lambda_0, lambda_1 = parameters

        # Set initial condition
        self.sim.set_state([initial_volume])

        # Set growth constants
        self.sim.set_constant('central.lambda_0', lambda_0)
        self.sim.set_constant('central.lambda_1', lambda_1)

        # Define logged variable
        loggedVariable = 'central.volume_t'

        # Simulate
        output = self.sim.run(times[-1] + 1, log=[loggedVariable], log_times=times)
        result = output[loggedVariable]

        return np.array(result)

In [15]:
#
# Naive attempt to find an optimal parameter set psi for each mouse in the LXF A677 population.
#
# This cell needs above defined wrapped myokit model:
# [PintsModel]
#

import os

import numpy as np
import pandas as pd
import pints


# Import data
# Get path of current working directory
path = os.getcwd()

# Import LXF A677 control growth data
data = pd.read_csv(path + '/data/lxf_control_growth.csv')
n_mice = len(data['#ID'].unique())

# Define container for the structural model estimates
# Shape (n_mice, n_parameters)
n_parameters = 3
mouse_parameters = np.empty(shape=(n_mice, n_parameters))

# Define container for the objective function score for the optimised parameters
mouse_scores = np.empty(shape=n_mice)

# Define arbitrary starting point for the optimisations
initial_parameters = [1, 1, 1]

# Find mouse parameters for LXF A677 population
mouse_ids = data['#ID'].unique()
for index, mouse_id in enumerate(mouse_ids):
    # Create mask for mouse with specfied ID
    mouse_mask = data['#ID'] == mouse_id

    # Get relevant time points
    times = data[mouse_mask]['TIME in day'].to_numpy()

    # Get measured tumour volumes
    observed_volumes = data[mouse_mask]['TUMOUR VOLUME in cm^3'].to_numpy()

    # Create inverse problem
    problem = pints.SingleOutputProblem(PintsModel(), times, observed_volumes)

    # Create sum of squares error objective function
    error = pints.SumOfSquaresError(problem)

    # Create optimisation controller with a CMA-ES optimiser
    optimiser = pints.OptimisationController(
        function=error,
        x0=initial_parameters,
        method=pints.CMAES)

    # Disable logging mode
    optimiser.set_log_to_screen(False)

    # Parallelise optimisation
    optimiser.set_parallel(True)

    # Find optimal parameters
    estimates, score = optimiser.run()

    # Save estimates and score
    mouse_parameters[index, :] = estimates
    mouse_scores[index] = score

In [16]:
#
# Solve structural model for inferred model parameters.
#
# This cell needs the above defined wrapped myokit model, and the inferred model parameters:
# [PintsModel, mouse_parameters]
#

import numpy as np


# Create tumour growth model
model = PintsModel()

# Define simulation time points in day
times = np.linspace(start=0, stop=30, num=200)
n_times = len(times)

# Create container for simulated tumour growth
# Shape (n_mice, n_times)
n_mice = len(mouse_parameters)
tumour_growth = np.empty(shape=(n_mice, n_times))

# Solve structural model for LXF A677 population
for mouse_id, mouse_params in enumerate(mouse_parameters):
    # Simulate mouse tumour growth
    tumour_growth[mouse_id, :] = model.simulate(parameters=mouse_params, times=times)


In [17]:
#
# Visualise simulated tumour growth together with control growth data.
#
# This cell needs the above simulated tumour growth and the control growth data:
# [times, tumour_growth, data]
#

import pandas as pd
import plotly.colors
import plotly.graph_objects as go


# Get number of individual mice
n_mice = len(tumour_growth)

# Define colorscheme
colors = plotly.colors.qualitative.Plotly[:n_mice]

# Create figure
fig = go.Figure()

# Scatter plot LXF A677 time-series data for each mouse
mouse_ids = data['#ID'].unique()
for index, id_m in enumerate(mouse_ids):
    # Create mask for mouse
    mask = data['#ID'] == id_m

    # Get time points for mouse
    observed_times = data['TIME in day'][mask]

    # Get observed tumour volumes for mouse
    observed_volumes = data['TUMOUR VOLUME in cm^3'][mask]

    # Get simulated tumour volumes for mouse
    simulated_volumes = tumour_growth[index, :]

    # Plot data
    fig.add_trace(go.Scatter(
        x=observed_times,
        y=observed_volumes,
        legendgroup="ID: %d" % id_m,
        name="ID: %d" % id_m,
        showlegend=True,
        hovertemplate=
            "<b>Measurement </b><br>" +
            "ID: %d<br>" % id_m +
            "Time: %{x:} day<br>" +
            "Tumour volume: %{y:.02f} cm^3<br>" +
            "Cancer type: LXF A677<br>" +
            "<extra></extra>",
        mode="markers",
        marker=dict(
            symbol='circle',
            opacity=0.7,
            line=dict(color='black', width=1),
            color=colors[index])
    ))

    # Plot simulated growth
    fig.add_trace(go.Scatter(
        x=times,
        y=simulated_volumes,
        legendgroup="ID: %d" % id_m,
        name="ID: %d" % id_m,
        showlegend=False,
        hovertemplate=
            "<b>Simulation </b><br>" +
            "ID: %d<br>" % id_m +
            "Time: %{x:.0f} day<br>" +
            "Tumour volume: %{y:.02f} cm^3<br>" +
            "Cancer type: LXF A677<br>" +
            "<extra></extra>",
        mode="lines",
        line=dict(color=colors[index])
    ))

# Set X, Y axis and figure size
fig.update_layout(
    autosize=True,
    xaxis_title=r'$\text{Time in day}$',
    yaxis_title=r'$\text{Tumour volume in cm}^3$',
    template="plotly_white")

# Add switch between linear and log y-scale
fig.update_layout(
    updatemenus=[
        dict(
            type = "buttons",
            direction = "left",
            buttons=list([
                dict(
                    args=[{"yaxis.type": "linear"}],
                    label="Linear y-scale",
                    method="relayout"
                ),
                dict(
                    args=[{"yaxis.type": "log"}],
                    label="Log y-scale",
                    method="relayout"
                )
            ]),
            pad={"r": 0, "t": -10},
            showactive=True,
            x=0.0,
            xanchor="left",
            y=1.15,
            yanchor="top"
        ),
    ]
)

# Show figure
fig.show()

**Figure 1:** Naïve optimisation of the structural model for each mouse starting from an initial point $\psi _0=(1, 1, 1)$ in parameter space. We used the CMA-ES optimiser to minimise the squared distance between the model predictions and the observations. If the model was identifiable and the optimiser perfect, the inferred set of model parameters would be independent of the starting point of the optmisation $\psi _0$.

## Stability of optimisation

In PKPD modelling, we are interested in predicting efficacious, while safe dosing regimens for patients. Those predictions crucially depend on the accuracy of the estimated structural model parameters $\hat \psi $. This is especially true when we try to predict the effects of the drug outside of the measured time frame, or try to predict disease progression under alternative treatment strategies.

The stability of the inference of patient parameters can already be assessed by analysing the inferred parameters from the mouse data set. If the above inferred structural model parameters are trustworthy, running the same optimisation routine multiple times would yield the same set of parameters every time. To illustrate the sensitivity of numerical optimisers we will now choose a different starting point for each optimisation (uniform in $[10^{-3}, 10^3]$ for each parameter). Ideally we would like to see that each optimisation returns very similar model parameters for an individual. This would give us confidence to interpret those model parameters in a biological context.

In [45]:
#
# Run optimisation multiple times from random initial starting points.
#
# This cell needs above defined wrapped myokit model:
# [PintsModel]
#

import os

import myokit
import numpy as np
import pandas as pd
import pints


# Define number of optimisation runs for each individual
n_runs = 10

# Import data
# Get path of current working directory
path = os.getcwd()

# Import LXF A677 control growth data
data = pd.read_csv(path + '/data/lxf_control_growth.csv')
n_mice = len(data['#ID'].unique())

# Define container for the structural model estimates
# Shape (n_mice, n_runs, n_parameters)
n_parameters = 3
mouse_parameters_multi_runs = np.empty(shape=(n_mice, n_runs, n_parameters))

# Define container for the objective function score for the optimised parameters
mouse_scores_multi_runs = np.empty(shape=(n_mice, n_runs))

# Define random starting points over many orders of magnitude
# Shape = (n_runs, n_parameters)
initial_parameters = np.random.uniform(low=1E-3, high=1E3, size=(n_runs, n_parameters))

# Find mouse parameters for LXF A677 population
mouse_ids = data['#ID'].unique()
for index, mouse_id in enumerate(mouse_ids):
    # Create mask for mouse with specfied ID
    mouse_mask = data['#ID'] == mouse_id

    # Get relevant time points
    times = data[mouse_mask]['TIME in day'].to_numpy()

    # Get measured tumour volumes
    observed_volumes = data[mouse_mask]['TUMOUR VOLUME in cm^3'].to_numpy()

    # Create inverse problem
    problem = pints.SingleOutputProblem(PintsModel(), times, observed_volumes)

    # Create sum of squares error objective function
    error = pints.SumOfSquaresError(problem)

    # Run optimisation multiple times
    for run_id, initial_params in enumerate(initial_parameters):
        # Create optimisation controller with a CMA-ES optimiser
        optimiser = pints.OptimisationController(
            function=error,
            x0=initial_params,
            method=pints.CMAES)

        # Disable logging mode
        optimiser.set_log_to_screen(False)

        # Parallelise optimisation
        optimiser.set_parallel(True)

        # Find optimal parameters
        try:
            estimates, score = optimiser.run()
        except:
            # If inference breaks fill estimates with nan
            estimates = np.array([np.nan, np.nan, np.nan])
            score = np.nan

        # Save estimates and score
        mouse_parameters_multi_runs[index, run_id, :] = estimates
        mouse_scores_multi_runs[index, run_id] = score


----------------------------------------
Unexpected termination.
Current best score: 0.5354064517406236
Current best position:
-1.51301662058000206e-05
 8.60852454032899118e+02
 8.85105664970445016e-02
----------------------------------------


The unexpected termination message indicates that we ran into optimisation errors for at least one run. Let us explore the parameters from the successfull optimisations below.

In [56]:
#
# Visualisation of the spread of optimised model parameters for multiple runs from different initial points.
#
# This cell needs the above optimised model parameters and their respective objective function scores, as well as the data
# [mouse_parameters_multi_runs, mouse_scores_multi_runs, data]
#

import plotly.colors
import plotly.graph_objects as go


# Get mouse ids
mouse_ids = data['#ID'].unique()

# Get number of parameters + score (for visualisation)
n_params = mouse_parameters_multi_runs.shape[2] + 1

# Define colorscheme
colors = plotly.colors.qualitative.Plotly[:n_params]

# Get optimised parameter sets
optimised_parameters =  mouse_parameters_multi_runs

# Get scores for parameters
scores = mouse_scores_multi_runs

# Create figure
fig = go.Figure()

# Box plot of optimised model parameters
for index, id_m in enumerate(mouse_ids):
    # Get optimised parameters
    parameters = optimised_parameters[index, ...]

    # Get scores
    score = scores[index, :]

    # Create box plot of for initial tumour volume
    fig.add_trace(
        go.Box(
            y=parameters[:, 0],  
            name="Initial tumour volume in cm^3",
            boxpoints='all',
            jitter=0.2,
            pointpos=-1.5,
            visible=True if index == 0 else False,
            marker=dict(
                symbol='circle',
                opacity=0.7,
                line=dict(color='black', width=1)),
            marker_color=colors[0],
            line_color=colors[0]))

    # Create box plot of for exponential tumour growth
    fig.add_trace(
        go.Box(
            y=parameters[:, 1],  
            name="Exponential growth rate in 1/day",
            boxpoints='all',
            jitter=0.2,
            pointpos=-1.5,
            visible=True if index == 0 else False,
            marker=dict(
                symbol='circle',
                opacity=0.7,
                line=dict(color='black', width=1)),
            marker_color=colors[1],
            line_color=colors[1]))

    # Create box plot of for linear tumour growth
    fig.add_trace(
        go.Box(
            y=parameters[:, 2],  
            name="Linear growth rate in cm^3/day",
            boxpoints='all',
            jitter=0.2,
            pointpos=-1.5,
            visible=True if index == 0 else False,
            marker=dict(
                symbol='circle',
                opacity=0.7,
                line=dict(color='black', width=1)),
            marker_color=colors[2],
            line_color=colors[2]))
    
    # Create box plot of for objective function score
    fig.add_trace(
        go.Box(
            y=score,  
            name="Score",
            boxpoints='all',
            jitter=0.2,
            pointpos=-1.5,
            visible=True if index == 0 else False,
            marker=dict(
                symbol='circle',
                opacity=0.7,
                line=dict(color='black', width=1)),
            marker_color=colors[3],
            line_color=colors[3]))

# Set figure size
fig.update_layout(
    autosize=True,
    template="plotly_white")

# Add switch between mice
fig.update_layout(
    updatemenus=[
        dict(
            type = "buttons",
            direction = "right",
            buttons=list([
                dict(
                    args=[{"visible": [True]*4 + [False]*(4 * 7)}],
                    label="ID: %d" % mouse_ids[0],
                    method="restyle"
                ),
                dict(
                    args=[{"visible": [False]*4 + [True]*4 + [False]*(4 * 6)}],
                    label="ID: %d" % mouse_ids[1],
                    method="restyle"
                ),
                dict(
                    args=[{"visible": [False]*(4 * 2) + [True]*4 + [False]*(4 * 5)}],
                    label="ID: %d" % mouse_ids[2],
                    method="restyle"
                ),
                dict(
                    args=[{"visible": [False]*(4 * 3) + [True]*4 + [False]*(4 * 4)}],
                    label="ID: %d" % mouse_ids[3],
                    method="restyle"
                ),
                dict(
                    args=[{"visible": [False]*(4 * 4) + [True]*4 + [False]*(4 * 3)}],
                    label="ID: %d" % mouse_ids[4],
                    method="restyle"
                ),
                dict(
                    args=[{"visible": [False]*(4 * 5) + [True]*4 + [False]*(4 * 2)}],
                    label="ID: %d" % mouse_ids[5],
                    method="restyle"
                ),
                dict(
                    args=[{"visible": [False]*(4 * 6) + [True]*4 + [False]* 4}],
                    label="ID: %d" % mouse_ids[6],
                    method="restyle"
                ),
                dict(
                    args=[{"visible": [False]*(4 * 7) + [True]*4}],
                    label="ID: %d" % mouse_ids[7],
                    method="restyle"
                )
            ]),
            pad={"r": 0, "t": -10},
            showactive=True,
            x=0.0,
            xanchor="left",
            y=1.1,
            yanchor="top"
        )
    ]
)

# Position legend
fig.update_layout(legend=dict(
    yanchor="bottom",
    y=0.01,
    xanchor="left",
    x=1.05))

# Show figure
fig.show()


**Figure 2:** Scatter and box plot of the structural model parameters $\psi $ (initial tumour volume $V_0$, exponential growth rate $\lambda _0$, linear growth rate $\lambda _1$) found by minimising the squared distance between the predictions and the observations using a CMA-ES optimiser. For each individual the optimisation routine was run 10 times from a uniformly sampled starting point in $[10^{-3}, 10^3]$. In addition to the optimised parameters, also the distribution of the associated objective function scores is presented.

Running the same optimisation routine multiple times reveals that the optimiser has a hard time to find the globally optimal set of model parameters $\psi $, see Figure 2. Especially the exponential growth rate $\lambda _0$ and the linear growth rate $\lambda _1$ appear to vary significantly between optimisations for all individuals. 

The reasons why an optimiser, such as CMA-ES, may fail to produce the same estimates when running the optimisation multiple times, are manifold, but may be grouped into model-specific and optimiser-specific reasons. 

An optimiser might fail to return a consistent set of model parameters, because the model may produce identical predictions for distinct sets of model parameters $\psi $ and $\psi'$. If those parameters happen to also minimise the objective function $L$, the optimiser returns either $\psi $ or $\psi '$. In other words, if the objective function $L$ cannot distinguish between two parameter sets, because the predictions of the model are identical $V^s_T(\psi) = V^s_T(\psi')$, although the parameters are not $\psi \neq \psi'$, the optimiser will return either parameter set. In this scenario the model is said to be non-identifiable. Non-identifiability may either occur because the model structure does not allow unambiguous parameter identification (structural non-identifiability), or the data is not sufficient to reveal the relevance of a subset of parameters (practical non-identifiability) [6].

On the other hand, an optimiser might fail to find the globally optimal parameter set of an identifiable model, because of limitations of the optimisation routine itself.  

## Non-dimensionalisation of the model

One of the simplest strategies to avoid a structural non-identifiability of a model is to transform the model parameters $\psi $ from a biologically meaningful set of parameters $(V_0, \lambda _0, \lambda _1)$ to a set of dimensionless model parameters. This can be done by introducing characteristic scales of the problem, such as a characteristic tumour volume $V^c_T$ and a characteristic time $t^c$. If we express the structural predictions of the tumour volume in units of the characteristic tumour volume $v = V^s_T / V^c_T$ and the time in units of the characteristic time $\tau = t / t^c $, we can rewrite the structural model in a nondimensional form

\begin{equation*}
   \frac{\text{d}v}{\text{d}\tau} = \frac{a_1 v}{v + a_0},
\end{equation*}

where $a_0 = \lambda _1 / 2 \lambda _0 V^c_T$ and $a_1 = \lambda _1 t^c / V^c_T$. In other words, we used the characteristic volume and time scales to express the model parameters in units of them

\begin{equation*}
   (V_0, \lambda _0, \lambda _1) = \left( v_0\, V^c_T, \frac{a_1}{a_0}\frac{1}{t^c}, a_1 \frac{V^c_T}{t^c} \right) .
\end{equation*}

Here, $(v_0, a_0, a_1)$ take the role of the model parameters. Arguably for this biological process a characteristic volume scale is $1\, \text{cm}^3$ and a characteristic time scale is $1\, \text{day}$.


In [1]:
#
# Create structural model.
#

import pkpd.model as m


# Create model
model = m.create_dimless_tumour_growth_model()

# Show model
print(model.code())

[[model]]
# Initial values
central.volume_t = 0

[central]
a_0 = 1
    in [1]
a_1 = 0
    in [1]
time = 0 bind time
    in [1]
dot(volume_t) = a_1 * volume_t / (volume_t + a_0)
    in [1]




In [2]:
#
# Define pints model wrapper such that myokit model can be used for inference.
#

import myokit
import pints

from pkpd import model as model


# Wrap myokit model, so it can be used with pints
class DimensionlessPintsModel(pints.ForwardModel):
    def __init__(self):
        # Create myokit model
        model = m.create_dimless_tumour_growth_model()

        # Create simulator
        self.sim = myokit.Simulation(model)

    def n_parameters(self):
        """
        Number of parameters to fit. Here initial v, a_0, a_1.
        """
        return 3

    def n_outputs(self):
        return 1

    def simulate(self, parameters, times):
        # Reset simulation
        self.sim.reset()

        # Sort input parameters
        initial_volume, a_0, a_1 = parameters

        # Set initial condition
        self.sim.set_state([initial_volume])

        # Set growth constants
        self.sim.set_constant('central.a_0', a_0)
        self.sim.set_constant('central.a_1', a_1)

        # Define logged variable
        loggedVariable = 'central.volume_t'

        # Simulate
        output = self.sim.run(times[-1] + 1, log=[loggedVariable], log_times=times)
        result = output[loggedVariable]

        return np.array(result)

In [3]:
#
# Run optimisation multiple times from random initial starting points. 
#
# Characteristic scale were chosen to be: V^c_T = 1 cm^3, t^c = 1 day.
#
# This cell needs above defined wrapped myokit model:
# [DimensionlessPintsModel]
#

import os

import myokit
import numpy as np
import pandas as pd
import pints


# Define characteristic scales
characteristic_volume = 1  # in cm^3
characteristic_time = 1  # in day

# Define number of optimisation runs for each individual
n_runs = 10

# Import data
# Get path of current working directory
path = os.getcwd()

# Import LXF A677 control growth data
data = pd.read_csv(path + '/data/lxf_control_growth.csv')
n_mice = len(data['#ID'].unique())

# Define container for the structural model estimates
# Shape (n_mice, n_runs, n_parameters)
n_parameters = 3
mouse_parameters_multi_runs_dimless = np.empty(shape=(n_mice, n_runs, n_parameters))

# Define container for the objective function score for the optimised parameters
mouse_scores_multi_runs_dimless = np.empty(shape=(n_mice, n_runs))

# Define random starting points over many orders of magnitude
# Shape = (n_runs, n_parameters)
initial_parameters = np.random.uniform(low=1E-3, high=1E3, size=(n_runs, n_parameters))

# Find mouse parameters for LXF A677 population
mouse_ids = data['#ID'].unique()
for index, mouse_id in enumerate(mouse_ids):
    # Create mask for mouse with specfied ID
    mouse_mask = data['#ID'] == mouse_id

    # Get relevant time points
    times = data[mouse_mask]['TIME in day'].to_numpy() / characteristic_time

    # Get measured tumour volumes
    observed_volumes = data[mouse_mask]['TUMOUR VOLUME in cm^3'].to_numpy() / characteristic_volume

    # Create inverse problem
    problem = pints.SingleOutputProblem(DimensionlessPintsModel(), times, observed_volumes)

    # Create sum of squares error objective function
    error = pints.SumOfSquaresError(problem)

    # Run optimisation multiple times
    for run_id, initial_params in enumerate(initial_parameters):
        # Create optimisation controller with a CMA-ES optimiser
        optimiser = pints.OptimisationController(
            function=error,
            x0=initial_params,
            method=pints.CMAES)

        # Disable logging mode
        optimiser.set_log_to_screen(False)

        # Parallelise optimisation
        optimiser.set_parallel(True)

        # Find optimal parameters
        try:
            estimates, score = optimiser.run()
        except:
            # If inference breaks fill estimates with nan
            estimates = np.array([np.nan, np.nan, np.nan])
            score = np.nan

        # Save estimates and score
        mouse_parameters_multi_runs_dimless[index, run_id, :] = estimates
        mouse_scores_multi_runs_dimless[index, run_id] = score


----------------------------------------
Unexpected termination.
Current best score: 4.313994139950808
Current best position:
-1.55166849151601127e+00
 8.29856227814939530e+02
-9.86147709376575676e+02
----------------------------------------


In [10]:
#
# Transform dimensionless parameters back to biological parameters.
# 
# This cell needs the above inferred dimensionless parameters and the characteristic volume and time scale:
# [mouse_parameters_multi_runs_dimless, characteristic_volume, characteristic_time]
#

import numpy as np


# Initialise container for backtransformed paramters
# Shape (n_mice, n_runs, n_parameters)
mouse_parameters_dimless_optimisation = np.empty(shape=mouse_parameters_multi_runs_dimless.shape)

# Transform initial volumes
mouse_parameters_dimless_optimisation[:, :, 0] = mouse_parameters_multi_runs_dimless[:, :, 0] * characteristic_volume

# Transform exponential growth rate
# lambda_0 = a_1 / a_0 /t^c
mouse_parameters_dimless_optimisation[:, :, 1] = \
    mouse_parameters_multi_runs_dimless[:, :, 2] / mouse_parameters_multi_runs_dimless[:, :, 1] / characteristic_time

# Transform linear growth rate
# lambda_1 = a_1 / a_0 /t^c
mouse_parameters_dimless_optimisation[:, :, 2] = \
    mouse_parameters_multi_runs_dimless[:, :, 2] * characteristic_volume / characteristic_time


In [11]:
#
# Visualisation of the spread of optimised model parameters for multiple runs from different initial points.
#
# This cell needs the above optimised initial parameter from psi_0=(1, 1, 1) and the five runs from a random initial starting point, and their respective objective function scores, as well as the data
# [mouse_parameters, mouse_parameters_multi_runs, mouse_score, mouse_scores_multi_runs, data]
#

import plotly.colors
import plotly.graph_objects as go


# Get mouse ids
mouse_ids = data['#ID'].unique()

# Get number of parameters + score (for visualisation)
n_params = mouse_parameters_multi_runs_dimless.shape[2] + 1

# Define colorscheme
colors = plotly.colors.qualitative.Plotly[:n_params]

# Get optimised parameter sets
optimised_parameters =  mouse_parameters_dimless_optimisation

# Get scores for parameters
scores = mouse_scores_multi_runs_dimless

# Create figure
fig = go.Figure()

# Box plot of optimised model parameters
for index, id_m in enumerate(mouse_ids):
    # Get optimised parameters
    parameters = optimised_parameters[index, ...]

    # Get scores
    score = scores[index, :]

    # Create box plot of for initial tumour volume
    fig.add_trace(
        go.Box(
            y=parameters[:, 0],  
            name="Initial tumour volume in cm^3",
            boxpoints='all',
            jitter=0.2,
            pointpos=-1.5,
            visible=True if index == 0 else False,
            marker=dict(
                symbol='circle',
                opacity=0.7,
                line=dict(color='black', width=1)),
            marker_color=colors[0],
            line_color=colors[0]))

    # Create box plot of for exponential tumour growth
    fig.add_trace(
        go.Box(
            y=parameters[:, 1],  
            name="Exponential growth rate in 1/day",
            boxpoints='all',
            jitter=0.2,
            pointpos=-1.5,
            visible=True if index == 0 else False,
            marker=dict(
                symbol='circle',
                opacity=0.7,
                line=dict(color='black', width=1)),
            marker_color=colors[1],
            line_color=colors[1]))

    # Create box plot of for linear tumour growth
    fig.add_trace(
        go.Box(
            y=parameters[:, 2],  
            name="Linear growth rate in cm^3/day",
            boxpoints='all',
            jitter=0.2,
            pointpos=-1.5,
            visible=True if index == 0 else False,
            marker=dict(
                symbol='circle',
                opacity=0.7,
                line=dict(color='black', width=1)),
            marker_color=colors[2],
            line_color=colors[2]))
    
    # Create box plot of for objective function score
    fig.add_trace(
        go.Box(
            y=score,  
            name="Score",
            boxpoints='all',
            jitter=0.2,
            pointpos=-1.5,
            visible=True if index == 0 else False,
            marker=dict(
                symbol='circle',
                opacity=0.7,
                line=dict(color='black', width=1)),
            marker_color=colors[3],
            line_color=colors[3]))

# Set figure size
fig.update_layout(
    autosize=True,
    template="plotly_white")

# Add switch between mice
fig.update_layout(
    updatemenus=[
        dict(
            type = "buttons",
            direction = "right",
            buttons=list([
                dict(
                    args=[{"visible": [True]*4 + [False]*(4 * 7)}],
                    label="ID: %d" % mouse_ids[0],
                    method="restyle"
                ),
                dict(
                    args=[{"visible": [False]*4 + [True]*4 + [False]*(4 * 6)}],
                    label="ID: %d" % mouse_ids[1],
                    method="restyle"
                ),
                dict(
                    args=[{"visible": [False]*(4 * 2) + [True]*4 + [False]*(4 * 5)}],
                    label="ID: %d" % mouse_ids[2],
                    method="restyle"
                ),
                dict(
                    args=[{"visible": [False]*(4 * 3) + [True]*4 + [False]*(4 * 4)}],
                    label="ID: %d" % mouse_ids[3],
                    method="restyle"
                ),
                dict(
                    args=[{"visible": [False]*(4 * 4) + [True]*4 + [False]*(4 * 3)}],
                    label="ID: %d" % mouse_ids[4],
                    method="restyle"
                ),
                dict(
                    args=[{"visible": [False]*(4 * 5) + [True]*4 + [False]*(4 * 2)}],
                    label="ID: %d" % mouse_ids[5],
                    method="restyle"
                ),
                dict(
                    args=[{"visible": [False]*(4 * 6) + [True]*4 + [False]* 4}],
                    label="ID: %d" % mouse_ids[6],
                    method="restyle"
                ),
                dict(
                    args=[{"visible": [False]*(4 * 7) + [True]*4}],
                    label="ID: %d" % mouse_ids[7],
                    method="restyle"
                )
            ]),
            pad={"r": 0, "t": -10},
            showactive=True,
            x=0.0,
            xanchor="left",
            y=1.1,
            yanchor="top"
        )
    ]
)

# Position legend
fig.update_layout(legend=dict(
    yanchor="bottom",
    y=0.01,
    xanchor="left",
    x=1.05))

# Show figure
fig.show()


**Figure 2:** Scatter and box plot of the structural model parameters $\psi $ (initial tumour volume $V_0$, exponential growth rate $\lambda _0$, linear growth rate $\lambda _1$) found by minimising the squared distance between the predictions and the observations using a CMA-ES optimiser. For each individual the optimisation routine was run 10 times from a uniformly sampled starting point in $[10^{-3}, 10^3]$. In addition to the optimised parameters, also the distribution of the associated objective function scores is presented.

## Log-transforming the model parameters

In [49]:
#
# Define log-tranformed pints model wrapper such that myokit model can be used for inference. In contrast to the PintsModel above, this model expects log-transformed parameters log(psi).
#

import myokit
import numpy as np
import pints

from pkpd import model as model


# Wrap myokit model, so it can be used with pints
class LogTransformedPintsModel(pints.ForwardModel):
    def __init__(self):
        # Create myokit model
        model = m.create_tumour_growth_model()

        # Create simulator
        self.sim = myokit.Simulation(model)

    def n_parameters(self):
        """
        Number of parameters to fit. Here initial log(V^s_T), log(lambda_0), log(lambda_1)
        """
        return 3

    def n_outputs(self):
        return 1

    def simulate(self, log_parameters, times):
        # Reset simulation
        self.sim.reset()

        # Sort input parameters and transform to linear scale
        initial_volume, lambda_0, lambda_1 = np.exp(parameters)

        # Set initial condition
        self.sim.set_state([initial_volume])

        # Set growth constants
        self.sim.set_constant('central.lambda_0', lambda_0)
        self.sim.set_constant('central.lambda_1', lambda_1)

        # Define logged variable
        loggedVariable = 'central.volume_t'

        # Simulate
        output = self.sim.run(times[-1] + 1, log=[loggedVariable], log_times=times)
        result = output[loggedVariable]

        return np.array(result)

In [52]:
#
# Run optimisation multiple times from random initial starting points.
#
# This cell needs above defined wrapped myokit model:
# [LogTransformedPintsModel]
#

import os

import myokit
import numpy as np
import pandas as pd
import pints


# Define number of optimisation runs for each individual
n_runs = 10

# Import data
# Get path of current working directory
path = os.getcwd()

# Import LXF A677 control growth data
data = pd.read_csv(path + '/data/lxf_control_growth.csv')
n_mice = len(data['#ID'].unique())

# Define container for the structural model estimates
# Shape (n_mice, n_runs, n_parameters)
n_parameters = 3
mouse_parameters_multi_runs_log_transformed = np.empty(shape=(n_mice, n_runs, n_parameters))

# Define container for the objective function score for the optimised parameters
mouse_scores_multi_runs_log_transformed = np.empty(shape=(n_mice, n_runs))

# Define random starting points over many orders of magnitude
# Shape = (n_runs, n_parameters)
initial_parameters = np.random.uniform(low=1E-3, high=1E3, size=(n_runs, n_parameters))

# Find mouse parameters for LXF A677 population
mouse_ids = data['#ID'].unique()
for index, mouse_id in enumerate(mouse_ids):
    # Create mask for mouse with specfied ID
    mouse_mask = data['#ID'] == mouse_id

    # Get relevant time points
    times = data[mouse_mask]['TIME in day'].to_numpy()

    # Get measured tumour volumes
    observed_volumes = data[mouse_mask]['TUMOUR VOLUME in cm^3'].to_numpy()

    # Create inverse problem
    problem = pints.SingleOutputProblem(PintsModel(), times, observed_volumes)

    # Create sum of squares error objective function
    error = pints.SumOfSquaresError(problem)

    # Run optimisation multiple times
    for run_id, initial_params in enumerate(initial_parameters):
        # Transform parameters to log-scale
        log_initial_params = np.log(initial_params)

        # Create optimisation controller with a CMA-ES optimiser
        optimiser = pints.OptimisationController(
            function=error,
            x0=log_initial_params,
            method=pints.CMAES)

        # Disable logging mode
        optimiser.set_log_to_screen(False)

        # Parallelise optimisation
        optimiser.set_parallel(True)

        # Find optimal parameters
        try:
            estimates, score = optimiser.run()
        except:
            # If inference breaks fill estimates with nan
            estimates = np.array([np.nan, np.nan, np.nan])
            score = np.nan

        # Save estimates and score (back transformed to linear scale)
        mouse_parameters_multi_runs_log_transformed[index, run_id, :] = np.exp(estimates)
        mouse_scores_multi_runs_log_transformed[index, run_id] = score


----------------------------------------
Unexpected termination.
Current best score: 1.2440895517828978
Current best position:
 4.36108352070969105e-04
 9.93611261526420364e+00
 5.05644705743254139e-02
----------------------------------------

----------------------------------------
Unexpected termination.
Current best score: 0.027833585817775376
Current best position:
 3.38636783865060298e-02
 5.27583230185733587e+00
 2.41953186218902654e-02
----------------------------------------

----------------------------------------
Unexpected termination.
Current best score: 66.29333568234948
Current best position:
 3.74305712437897808e+00
 8.23191641772539029e+00
-1.10531137144588551e-01
----------------------------------------


In [57]:
#
# Visualisation of the spread of optimised model parameters for multiple runs from different initial points.
#
# This cell needs the above optimised initial parameter from psi_0=(1, 1, 1) and the five runs from a random initial starting point, and their respective objective function scores, as well as the data
# [mouse_parameters_multi_runs_log_transformed, mouse_scores_multi_runs_log_transformed, data]
#

import plotly.colors
import plotly.graph_objects as go


# Get mouse ids
mouse_ids = data['#ID'].unique()

# Get number of parameters + score (for visualisation)
n_params = mouse_parameters.shape[1] + 1

# Define colorscheme
colors = plotly.colors.qualitative.Plotly[:n_params]

# Get optimised parameters
optimised_parameters = mouse_parameters_multi_runs_log_transformed

# Get optimised parameters
scores = mouse_scores_multi_runs_log_transformed

# Create figure
fig = go.Figure()

# Box plot of optimised model parameters
for index, id_m in enumerate(mouse_ids):
    # Get optimised parameters
    parameters = optimised_parameters[index, ...]

    # Get scores
    score = scores[index, :]

    # Create box plot of for initial tumour volume
    fig.add_trace(
        go.Box(
            y=parameters[:, 0],  
            name="Initial tumour volume in cm^3",
            boxpoints='all',
            jitter=0.2,
            pointpos=-1.5,
            visible=True if index == 0 else False,
            marker=dict(
                symbol='circle',
                opacity=0.7,
                line=dict(color='black', width=1)),
            marker_color=colors[0],
            line_color=colors[0]))

    # Create box plot of for exponential tumour growth
    fig.add_trace(
        go.Box(
            y=parameters[:, 1],  
            name="Exponential growth rate in 1/day",
            boxpoints='all',
            jitter=0.2,
            pointpos=-1.5,
            visible=True if index == 0 else False,
            marker=dict(
                symbol='circle',
                opacity=0.7,
                line=dict(color='black', width=1)),
            marker_color=colors[1],
            line_color=colors[1]))

    # Create box plot of for linear tumour growth
    fig.add_trace(
        go.Box(
            y=parameters[:, 2],  
            name="Linear growth rate in cm^3/day",
            boxpoints='all',
            jitter=0.2,
            pointpos=-1.5,
            visible=True if index == 0 else False,
            marker=dict(
                symbol='circle',
                opacity=0.7,
                line=dict(color='black', width=1)),
            marker_color=colors[2],
            line_color=colors[2]))
    
    # Create box plot of for objective function score
    fig.add_trace(
        go.Box(
            y=score,  
            name="Score",
            boxpoints='all',
            jitter=0.2,
            pointpos=-1.5,
            visible=True if index == 0 else False,
            marker=dict(
                symbol='circle',
                opacity=0.7,
                line=dict(color='black', width=1)),
            marker_color=colors[3],
            line_color=colors[3]))

# Set figure size
fig.update_layout(
    autosize=True,
    template="plotly_white")

# Add switch between mice
fig.update_layout(
    updatemenus=[
        dict(
            type = "buttons",
            direction = "right",
            buttons=list([
                dict(
                    args=[{"visible": [True]*4 + [False]*(4 * 7)}],
                    label="ID: %d" % mouse_ids[0],
                    method="restyle"
                ),
                dict(
                    args=[{"visible": [False]*4 + [True]*4 + [False]*(4 * 6)}],
                    label="ID: %d" % mouse_ids[1],
                    method="restyle"
                ),
                dict(
                    args=[{"visible": [False]*(4 * 2) + [True]*4 + [False]*(4 * 5)}],
                    label="ID: %d" % mouse_ids[2],
                    method="restyle"
                ),
                dict(
                    args=[{"visible": [False]*(4 * 3) + [True]*4 + [False]*(4 * 4)}],
                    label="ID: %d" % mouse_ids[3],
                    method="restyle"
                ),
                dict(
                    args=[{"visible": [False]*(4 * 4) + [True]*4 + [False]*(4 * 3)}],
                    label="ID: %d" % mouse_ids[4],
                    method="restyle"
                ),
                dict(
                    args=[{"visible": [False]*(4 * 5) + [True]*4 + [False]*(4 * 2)}],
                    label="ID: %d" % mouse_ids[5],
                    method="restyle"
                ),
                dict(
                    args=[{"visible": [False]*(4 * 6) + [True]*4 + [False]* 4}],
                    label="ID: %d" % mouse_ids[6],
                    method="restyle"
                ),
                dict(
                    args=[{"visible": [False]*(4 * 7) + [True]*4}],
                    label="ID: %d" % mouse_ids[7],
                    method="restyle"
                )
            ]),
            pad={"r": 0, "t": -10},
            showactive=True,
            x=0.0,
            xanchor="left",
            y=1.1,
            yanchor="top"
        )
    ]
)

# Position legend
fig.update_layout(legend=dict(
    yanchor="bottom",
    y=0.01,
    xanchor="left",
    x=1.05))

# Show figure
fig.show()


## Non-dimensional model with log-transformed parameters

In [ ]:
#
# Define pints model wrapper such that myokit model can be used for inference.
#

import myokit
import pints

from pkpd import model as model


# Wrap myokit model, so it can be used with pints
class DimensionlessPintsModel(pints.ForwardModel):
    def __init__(self):
        # Create myokit model
        model = m.create_dimless_tumour_growth_model()

        # Create simulator
        self.sim = myokit.Simulation(model)

    def n_parameters(self):
        """
        Number of parameters to fit. Here initial v, a_0, a_1.
        """
        return 3

    def n_outputs(self):
        return 1

    def simulate(self, parameters, times):
        # Reset simulation
        self.sim.reset()

        # Sort input parameters
        initial_volume, a_0, a_1 = parameters

        # Set initial condition
        self.sim.set_state([initial_volume])

        # Set growth constants
        self.sim.set_constant('central.a_0', a_0)
        self.sim.set_constant('central.a_1', a_1)

        # Define logged variable
        loggedVariable = 'central.volume_t'

        # Simulate
        output = self.sim.run(times[-1] + 1, log=[loggedVariable], log_times=times)
        result = output[loggedVariable]

        return np.array(result)

## Bibliography

- <a name="ref1"> [1] </a> Eigenmann et. al., Combining Nonclinical Experiments with Translational PKPD Modeling to Differentiate Erlotinib and Gefitinib, Mol Cancer Ther (2016)
- <a name="ref2"> [2] </a> Bellman, R. & Åström, K., On structural identifiability.Mathematical Biosciences7, 329 – 339 (1970)
- <a name="ref2"> [3] </a> Janzén, D. L. I. et al., Parameter identifiability of fundamental pharmacodynamic models.Frontiers in Physiology7, 590 (2016)
- <a name="ref2"> [4] </a> Lavielle, M. & Aarons, L., What do we mean by identifiability in mixed effects models?Journal of Pharmacoki-netics and Pharmacodynamics43, 111–122 (2016)
- <a name="ref2"> [5] </a> Hansen, N & Müller, S. D. & Koumoutsakos, P., Reducing the Time Complexity of the Derandomized Evolution Strategy with Covariance Matrix Adaptation (CMA-ES), Evolutionary Computation, 1-18 (2003)
- <a name="ref2"> [6] </a> Raue, A. et al., Structural and practical identifiability analysis of partially observed dynamical models by ex-ploiting the profile likelihood, Bioinformatics 25, 1923–1929 (2009)